In [10]:
import sys, os
import rasterio, geojson, folium, shapely

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely import geometry, wkt
from shapely.geometry import Point
from folium import plugins

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

In [3]:
input_folder = "/home/wb411133/data/Projects/FDP_Geospatial/AOI/CHAD"
in_camps = os.path.join(input_folder, "selected_camps.csv")

buildings_folder = "/home/public/Data/GLOBAL/Buildings/"
global_urban = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_SMOD/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0.tif"
global_pop   = "/home/public/Data/GLOBAL/Population/GHS/GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0.tif"

# define local files
local_urban = os.path.join(input_folder, "GHS_SMOD.tif")
local_pop = os.path.join(input_folder, "GHS_POP.tif")

In [33]:
inD = pd.read_csv(in_camps)
all_geom = []
for idx, row in inD.iterrows():    
    try:
        all_geom.append(Point(float(row['Lat']), float(row['Long'])))
    except:
        all_geom.append(None)

inD = gpd.GeoDataFrame(inD, geometry = all_geom, crs=4326)

In [65]:
# buffer the selected camps
inD = inD.to_crs(3857)
inD['geometry'] = inD['geometry'].buffer(2500)
inD = inD.to_crs(4326)
inD.to_file(os.path.join(input_folder, "camps.geojson"), driver="GeoJSON")

In [66]:
# map selected zones
m = folium.Map(location=[inD.unary_union.centroid.y,inD.unary_union.centroid.x], zoom_start=6)

for idx, row in inD.iterrows():
    geo_h = folium.GeoJson(shapely.geometry.mapping(row['geometry']), 
                         style_function=lambda x: {'fillColor': 'red'})
    geo_h.add_to(m)

m

# Extract secondary data

In [38]:
''' The GOOGLE buildings do not seem to intersect CHAD for some reason

# Identify google footprints tiles
buildings_footprints = "/home/public/Data/GLOBAL/Buildings/google_tiles.geojson"
inF = gpd.read_file(buildings_footprints)

#Download intersecting buildings manually
selF = inF.loc[inF.intersects(inD.unary_union)]
sel_tiles = selF['tile_url'].values
for t in sel_tiles:
    print(f'wget {t}')
    
for t in sel_tiles:    
    print(f'gzip -d {os.path.basename(t)}')
    
b = inD.total_bounds
all_buildings = []

for tile in sel_tiles:
    cur_tile = os.path.join(buildings_folder, os.path.basename(tile)[:-3])
    curT = pd.read_csv(cur_tile)
    selT = curT.loc[curT['latitude'].between(b[1], b[3])]
    selT = selT.loc[selT['longitude'].between(b[0], b[2])]    
    all_buildings.append(selT)
    tPrint(f'{os.path.basename(tile)[:-3]}: {selT.shape}')
final= pd.concat(all_buildings)
tPrint(f'Total buildings: {final.shape}')

inB_geom = final['geometry'].apply(wkt.loads)
inB = gpd.GeoDataFrame(final, geometry=inB_geom, crs=4326)
inB.reset_index()

sindex = inB.sindex

inD['gBuildings'] = 0
for idx, row in inD.iterrows():
    potential_matches = list(sindex.intersection(row['geometry'].bounds))
    tPrint(f'{idx}: {len(potential_matches)}')
    selB = inB.iloc[potential_matches]
    exact_matches = selB.loc[selB.intersects(row['geometry'])]
    inD.loc[idx, 'gBuildings'] = exact_matches.shape[0]
'''

,tile_id,tile_url,size_mb,geometry
0,025,https://storage.googleapis.com/open-buildings-...,0.2,"POLYGON ((-16.53484 -38.79460, -10.61966 -39.4..."
1,04f,https://storage.googleapis.com/open-buildings-...,0.7,"POLYGON ((-10.61966 -16.26673, -5.05987 -16.47..."
2,05b,https://storage.googleapis.com/open-buildings-...,0.2,"POLYGON ((-16.53484 -10.18989, -10.61966 -10.4..."
3,093,https://storage.googleapis.com/open-buildings-...,21.0,"POLYGON ((-28.67315 9.34189, -22.61986 9.81930..."
4,095,https://storage.googleapis.com/open-buildings-...,14.2,"POLYGON ((-28.67315 14.59941, -22.61986 15.325..."


In [72]:
# Summarize DA buildings
da_buildings = os.path.join(input_folder, "CHAD_Refugee_camp_buildings.shp")
inDA = gpd.read_file(da_buildings)
da_sindex = inDA.sindex

In [73]:
inD['daBuildings'] = 0
if inD.crs != inDA.crs:
  inD = inD.to_crs(inDA.crs)  
for idx, row in inD.iterrows():
    potential_matches = list(da_sindex.intersection(row['geometry'].bounds))
    tPrint(f'{idx}: {len(potential_matches)}')
    if len(potential_matches) > 0:
        selB = inDA.iloc[potential_matches]
        exact_matches = selB.loc[selB.intersects(row['geometry'])]
        inD.loc[idx, 'daBuildings'] = exact_matches.shape[0]

13:23:41	0: 11865
13:23:42	1: 12820
13:23:42	2: 20261
13:23:43	3: 8644
13:23:43	4: 15842
13:23:43	5: 17683
13:23:44	6: 20793
13:23:44	7: 23165
13:23:45	8: 13164
13:23:45	9: 14915
13:23:45	10: 9776
13:23:46	11: 14608
13:23:46	12: 11177
13:23:46	13: 11162
13:23:47	14: 13619
13:23:47	15: 1553
13:23:47	16: 3914
13:23:47	17: 4154
13:23:47	18: 3446
13:23:47	19: 6185
13:23:48	20: 9176
13:23:48	21: 6401
13:23:48	22: 90998
13:23:50	23: 4133


In [75]:
inD.to_file(os.path.join(input_folder, "CAMPS_buildings.shp"))

<ipython-input-75-88178f60616d>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  inD.to_file(os.path.join(input_folder, "CAMPS_buildings.shp"))


# Summarize Google Buildings